In [ ]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [1]:
input_default_json_conf_file='cnn_emotion4_training_yolo_default.json';

In [2]:
#import numpy as np
#import pandas as pd
#import tensorflow as tf
#import datetime
import shutil
import json

In [3]:
import sys
sys.path.append('../library');

In [4]:
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
## Load json conf json file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

# Variable globales

In [6]:
## Seed for the random variables
seed_number=0;

## Dataset 
dataset_dir    = DATA['dataset_dir'];
dataset_name   = DATA['dataset_name'];

## Training hyperparameters
EPOCAS     = DATA["epochs"];
BATCH_SIZE = DATA["batch_size"];

## Model of network
## 'yolov8n-cls','yolov8s-cls','yolov8m-cls'
model_type = DATA["model_type"];

## Output
output_base_dir = DATA["output_base_dir"];


# Parametros de entrada

In [7]:
for n in range(len(sys.argv)):
    if sys.argv[n]=='--dataset-dir':
        dataset_dir=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-name':
        dataset_name=sys.argv[n+1];
    elif sys.argv[n]=='--model':
        model_type=sys.argv[n+1];
    elif sys.argv[n]=='--epochs':
        EPOCAS=int(sys.argv[n+1]);
    elif sys.argv[n]=='--batch-size':
        BATCH_SIZE=int(sys.argv[n+1]);
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];
        
print('        dataset_base_dir:',dataset_dir)
print('            dataset_name:',dataset_name)
print('              model_type:',model_type)
print('                  EPOCAS:',EPOCAS)
print('              BATCH_SIZE:',BATCH_SIZE)
print('         output_base_dir:',output_base_dir)

        dataset_base_dir: /media/fernando/Expansion/DATASET/TESE/PATIENT-RECOGNITION/PATIENT-IMAGES/perwi/dataset/train/
     dataset_labels_file: labels-emotion4-v1.csv
   dataset_base_test_dir: /media/fernando/Expansion/DATASET/TESE/PATIENT-RECOGNITION/PATIENT-IMAGES/perwi/dataset/test/
dataset_labels_test_file: labels-emotion4-v1.csv
            dataset_name: perwi
              model_type: mobilenet_v3
                  EPOCAS: 50
              BATCH_SIZE: 32
         output_base_dir: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/cnn_emotion4


# Set seed of random variables


In [8]:
np.random.seed(seed_number)
#tf.keras.utils.set_random_seed(seed_number);

# Creating output directory

In [12]:
output_dir = os.path.join(output_base_dir,dataset_name,'training_validation_holdout',model_type);

os.makedirs(output_dir,exist_ok = True);

# Create new model

In [13]:
import BodyEmotion4Lib.lib_yolo_model as myp

model, target_size = mpp.create_model(model_type=model_type,load_weights=False,file_of_weight='');


#myp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats.m'));


Loading architecture mobilenet_v3

        url: https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5
target_size: (224, 224)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1024)              1529968   
                                                                 
 dense (Dense)               (None, 4)                 4100      
                                                                 
Total params: 1,534,068
Trainable params: 4,100
Non-trainable params: 1,529,968
_________________________________________________________________


# Train and validation


In [15]:


# https://docs.ultralytics.com/modes/train/#train-settings
results = model.train(data=dataset_dir, epochs=100, imgsz=224,batch=1,save=True,exist_ok=True)

best_model_file=os.path.join('runs','classify','train','weights','best.pt');



Epoch 1/50
16/16 [==============================] - ETA: 0s - loss: 1.3147 - categorical_accuracy: 0.3810
Epoch 1: val_loss improved from inf to 1.16100, saving model to /media/fernando/Expansion/OUTPUTS/DOCTORADO2/cnn_emotion4/perwi/training_validation_holdout/mobilenet_v3/model.h5
16/16 [==============================] - 16s 850ms/step - loss: 1.3147 - categorical_accuracy: 0.3810 - val_loss: 1.1610 - val_categorical_accuracy: 0.4882
Epoch 2/50
16/16 [==============================] - ETA: 0s - loss: 1.1580 - categorical_accuracy: 0.4742
Epoch 2: val_loss improved from 1.16100 to 1.06395, saving model to /media/fernando/Expansion/OUTPUTS/DOCTORADO2/cnn_emotion4/perwi/training_validation_holdout/mobilenet_v3/model.h5
16/16 [==============================] - 10s 651ms/step - loss: 1.1580 - categorical_accuracy: 0.4742 - val_loss: 1.0639 - val_categorical_accuracy: 0.5276
Epoch 3/50
16/16 [==============================] - ETA: 0s - loss: 1.0456 - categorical_accuracy: 0.5198
Epoch 3: v

# Evaluate best model

In [16]:
# LOAD BEST MODEL to evaluate the performance of the model
model=model.load(best_model_file);


data_results=dict();

# Evaluate training

# Evaluate validation
metrics = model.val();
print('validation',metrics,"\n\n");
data_results['val_categorical_accuracy']=metrics.top1;

# Evaluate testing


# final all json
with open(os.path.join(output_dir,"training_data_results.json"), 'w') as f:
    json.dump(data_results, f,indent=4);

# final test txt
with open(os.path.join(output_dir,"results_testing.txt"), 'w') as f: 
    for key, value in results.items(): 
        f.write('%s=%s;\n' % (key, value));

#######

shutil.move('runs', os.path.join(output_dir,'runs'))

16/16 [==============================] - 9s 565ms/step - loss: 0.4786 - categorical_accuracy: 0.8373
training {'loss': 0.4786042273044586, 'categorical_accuracy': 0.8373016119003296} 


4/4 [==============================] - 1s 323ms/step - loss: 0.8832 - categorical_accuracy: 0.6535
validation {'loss': 0.8832107782363892, 'categorical_accuracy': 0.6535432934761047} 


9/9 [==============================] - 7s 774ms/step - loss: 1.0629 - categorical_accuracy: 0.5568
testing {'loss': 1.062915563583374, 'categorical_accuracy': 0.5567765831947327} 




In [17]:

#POSTNAME=str(int(results['accuracy']*100000));
#tmp_name='modelo_'+model_type+'_acc'+POSTNAME+'.h5';

tmp_name='model_'+model_type+'.h5';

os.rename(best_model_file,os.path.join(output_dir,tmp_name));